<a href="https://colab.research.google.com/github/giotaChita/Artificial-Intelligence/blob/main/exe5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
import csv
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score



In [ ]:

trainy=pd.read_csv('y_train.csv')
train=pd.read_csv('X_train.csv')
print(train)



       pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  pixel8  \
0           0       0       0       0       0       0       0       0       0   
1           0       0       0       0       0       0       0       0       0   
2           0       0       0       0       0       0       0       0       0   
3           0       0       0       0       0       0       0       0       0   
4           0       0       0       0       0       0       0       0       0   
...       ...     ...     ...     ...     ...     ...     ...     ...     ...   
37928       0       0       0       0       0       0       0       0       0   
37929       0       0       0       0       0       0       0       0       0   
37930       0       0       0       0       0       0       0       0       0   
37931       3      11       0       0       0       0       0       0       0   
37932       0       0       0       0       0       0       0       0       0   

       pixel9  ...  pixel10

In [ ]:
train.insert(1024, "label", list(trainy.label.values-1), True)
np.random.shuffle(train.values)
trainx=train.copy()

del trainx["label"]
sc = StandardScaler()
trainx = sc.fit_transform(trainx)
trainy=train.loc[:,"label"]
print(trainy)



0         0
1        22
2        24
3        20
4        13
         ..
37928    28
37929     6
37930    12
37931    28
37932    21
Name: label, Length: 37933, dtype: int64


In [ ]:
trainy=np.asarray(trainy)
ohe = OneHotEncoder(sparse=False)
trainy=trainy.reshape(len(trainy),1)
trainy = ohe.fit_transform(trainy)
print(trainy)
X_train, X_test, Y_train, Y_test = train_test_split(trainx,trainy, test_size=0.01)

[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
lay=int(input("How many layers do you want the network to have (min 2)? "))
methodopt=int(input("Which hyperparameter optimization method to use? (1 for seperate, 2 for random) "))
bestact=''
bestunits=[50 for k in range(lay)]
bestacc=0
if methodopt==1:

  act=('relu','sigmoid','tanh')

  for j in act:
    model=keras.models.Sequential()

    model.add(keras.layers.Flatten(input_dim=(1024)))

    for i in range((lay-2)):
        model.add(keras.layers.Dense(units=50,activation=j))

    model.add(keras.layers.Dense(units=29,activation='sigmoid'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy','mse','mae'])

    model.fit(X_train,Y_train,epochs=20,batch_size=200)

    y_pred=model.predict(X_test)
    pred = list()
    for i in range(len(y_pred)):
        pred.append(np.argmax(y_pred[i]))
    test = list()
    for i in range(len(Y_test)):
        test.append(np.argmax(Y_test[i]))
    acc=accuracy_score(test,pred)
    if acc>bestacc:
      bestacc=acc
      bestact=j


  nounits=(100,150,200,250,300)
  for j in nounits:
    model=keras.models.Sequential()

    model.add(keras.layers.Flatten(input_dim=(1024)))

    for i in range((lay-2)):
        model.add(keras.layers.Dense(units=j,activation=bestact))

    model.add(keras.layers.Dense(units=29,activation='sigmoid'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy','mse','mae'])

    model.fit(X_train,Y_train,epochs=20,batch_size=200)

    y_pred=model.predict(X_test)
    pred = list()
    for i in range(len(y_pred)):
        pred.append(np.argmax(y_pred[i]))
    test = list()
    for i in range(len(Y_test)):
        test.append(np.argmax(Y_test[i]))
    acc=accuracy_score(test,pred)
    if acc>bestacc:
      bestacc=acc
      bestunits=[j for k in range(lay)]


else:
  act=['relu','sigmoid','tanh']
  for rnt in range(100):
    tact=act[random.randrange(0,3)]
    nounits=[random.randrange(50,300) for k in range(lay-2)]

    model=keras.models.Sequential()

    model.add(keras.layers.Flatten(input_dim=(1024)))

    for i in nounits:
        model.add(keras.layers.Dense(units=i,activation=tact))

    model.add(keras.layers.Dense(units=29,activation='sigmoid'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy','mse','mae'])

    model.fit(X_train,Y_train,epochs=20,batch_size=200)

    y_pred=model.predict(X_test)
    pred = list()
    for i in range(len(y_pred)):
        pred.append(np.argmax(y_pred[i]))
    test = list()
    for i in range(len(Y_test)):
        test.append(np.argmax(Y_test[i]))
    acc=accuracy_score(test,pred)
    if acc>bestacc:
      bestacc=acc
      bestunits=nounits.copy()
      bestact=tact

model=keras.models.Sequential()

model.add(keras.layers.Flatten(input_dim=(1024)))

#print(bestact)

for i in bestunits:
    model.add(keras.layers.Dense(units=i,activation=bestact))

model.add(keras.layers.Dense(units=29,activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy','mse','mae'])

model.fit(X_train,Y_train,epochs=80,batch_size=200)

y_pred=model.predict(X_test)
pred = list()
for i in range(len(y_pred)):
    pred.append(np.argmax(y_pred[i]))
test = list()
for i in range(len(Y_test)):
    test.append(np.argmax(Y_test[i]))
acc=accuracy_score(test,pred)

print(acc,bestact,bestunits)

How many layers do you want the network to have (min 2)? 5
Which hyperparameter optimization method to use? (1 for seperate, 2 for random) 1
Epoch 1/20
188/188 [==============================] - 2s 7ms/step - loss: 3.0862 - accuracy: 0.1340 - mse: 0.2610 - mae: 0.4847
Epoch 2/20
188/188 [==============================] - 1s 7ms/step - loss: 2.5629 - accuracy: 0.2464 - mse: 0.2636 - mae: 0.4405
Epoch 3/20
188/188 [==============================] - 1s 7ms/step - loss: 2.2867 - accuracy: 0.3132 - mse: 0.2628 - mae: 0.4146
Epoch 4/20
188/188 [==============================] - 1s 6ms/step - loss: 2.1143 - accuracy: 0.3577 - mse: 0.2568 - mae: 0.3942
Epoch 5/20
188/188 [==============================] - 1s 7ms/step - loss: 1.9895 - accuracy: 0.3897 - mse: 0.2542 - mae: 0.3816
Epoch 6/20
188/188 [==============================] - 1s 6ms/step - loss: 1.8884 - accuracy: 0.4154 - mse: 0.2519 - mae: 0.3725
Epoch 7/20
188/188 [==============================] - 1s 7ms/step - loss: 1.8090 - accuracy

KeyboardInterrupt: ignored

In [ ]:
testX=pd.read_csv('X_test.csv')
testy=pd.read_csv('y_test.csv')

testX = sc.transform(testX)

testy=np.asarray(testy)
testy=testy.reshape(len(testy),1)
testy = ohe.transform(testy)

y_pred = model.predict(testX)
#Converting predictions to label
pred = list()
for i in range(len(y_pred)):
    pred.append(np.argmax(y_pred[i]))
#Converting one hot encoded test label to label
test = list()
for i in range(len(testy)):
    test.append(np.argmax(testy[i]))
print(pred)
print(test)


[8, 0, 0, 0, 0, 0, 0, 28, 0, 0, 0, 0, 0, 0, 0, 0, 0, 28, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 12, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 27, 0, 0, 0, 0, 24, 0, 0, 0, 0, 0, 8, 0, 0, 0, 0, 0, 24, 0, 0, 0, 0, 0, 28, 0, 0, 0, 25, 0, 0, 0, 22, 0, 17, 8, 0, 3, 17, 0, 0, 0, 0, 0, 0, 27, 0, 28, 0, 0, 0, 0, 0, 0, 19, 0, 0, 4, 0, 0, 0, 0, 0, 3, 0, 0, 17, 0, 27, 0, 0, 0, 8, 0, 0, 28, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 28, 8, 0, 0, 0, 0, 0, 0, 17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 28, 0, 0, 0, 0, 0, 18, 22, 0, 0, 0, 8, 28, 0, 0, 0, 0, 0, 12, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 28, 0, 15, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 28, 0, 0, 28, 0, 0, 0, 0, 0, 0, 23, 22, 0, 0, 0, 0, 22, 0, 0, 0, 0, 0, 0, 6, 0, 0, 28, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 28, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 22, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 28, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 10, 28, 0, 0, 28, 0, 0

In [ ]:
# evaluation : 
#confusion matrix
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
#print(confusion_matrix(test, pred))
#plot_confusion_matrix(model,testX, testy)
#plt.show()

#precision
from sklearn.metrics import precision_score

print(f"Precision Score of the classifier is: {precision_score(test, pred,average='macro')}")


# recall
from sklearn.metrics import recall_score

# Calculating the recall score of classifier
print(f"Recall Score of the classifier is: {recall_score(test, pred,average='macro')}")

Precision Score of the classifier is: 0.947387242856312
Recall Score of the classifier is: 0.9464504536485254
